In [30]:
import pandas as pd
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import re
import json
import math
import os
from scipy.sparse import csr_matrix

In [31]:
# PATHS

stopwords_path = 'dataset/stopword-id.csv'
acronym_path = 'dataset/acronym.csv'
out_dir = "docs"

# Actual news data. Suit your need
csv_path = "../data/final_merge_dataset.csv"

In [32]:
# Load stopwords
stopwords_df = pd.read_csv(stopwords_path, header=None)
custom_stopwords = stopwords_df[0].tolist()
custom_stopwords +=['baiknya', 'berkali', 'kali', 'kurangnya', 'mata', 'olah', 'sekurang', 'setidak', 'tama', 'tidaknya']

# Load acronyms and build replacement dictionary
df_acronym = pd.read_csv(acronym_path)
acronym_dict = dict(zip(df_acronym["acronym"], df_acronym["expansion"]))
acronym_pattern = re.compile(r'\b(' + '|'.join(map(re.escape, acronym_dict.keys())) + r')\b')

# Text preprocessing function
def preprocess_text(text):
    text = str(text)
    
    # Replace acronyms
    text = acronym_pattern.sub(lambda match: acronym_dict[match.group(0)], text)
    
    # Lowercase
    text = text.lower()
    
    # Remove HTML image tags
    text = re.sub(r'<img[^>]*>', '', text)
    
    # Remove mentions, URLs, numbers
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'\d+', '', text)
    
    # Clean punctuation and excess whitespace
    text = text.replace("b'", "").replace("-", " ")
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Remove specific unwanted terms
    text = text.replace("img", "").replace("src", "")
    
    return text

In [33]:
# Load CSV into DataFrame
df = pd.read_csv(csv_path)

In [34]:
# PREPROCESS

df["preprocessed_text"] = df["Judul"].apply(preprocess_text)

In [36]:
# VECTORIZE

vectorizer = TfidfVectorizer(stop_words=custom_stopwords, max_features=1000)
vec = vectorizer.fit_transform(df['preprocessed_text'])

In [37]:
print(vec.shape)

(80472, 1000)


In [38]:
# Inputs
N, DIM = vec.shape
BYTES_PER_VEC = DIM * 4
TARGET_SHARD_BYTES = 32 * 1024 * 1024  # 32 MB target
vecs_per_shard = max(1, TARGET_SHARD_BYTES // BYTES_PER_VEC)


os.makedirs(out_dir, exist_ok=True)

manifest = {
    "total_vectors": int(N),
    "dim": int(DIM),
    "dtype": "float32",
    "header_size": 16,
    "shards": []
}

def write_header(f, num_vectors, dim, dtype_code=1):
    # magic + num_vectors + dim + dtype_code, all little-endian uint32
    f.write(b'VECT')  # 4 bytes
    f.write(np.uint32(num_vectors).tobytes())
    f.write(np.uint32(dim).tobytes())
    f.write(np.uint32(dtype_code).tobytes())

start = 0
shard_idx = 0
while start < N:
    end = min(N, start + vecs_per_shard)
    count = end - start
    shard_name = f"vectors_{shard_idx:03d}.f32"
    shard_path = os.path.join(out_dir, shard_name)
    print(f"Writing shard {shard_idx}: vectors {start}..{end-1} -> {shard_name} ({count} vectors)")
    with open(shard_path, "wb") as f:
        write_header(f, count, DIM, dtype_code=1)
        # Convert only this slice to dense float32 (row-wise)
        # For memory safety, do it in smaller sub-batches if needed
        batch_size = 1024  # adjust if your memory is tight
        for bstart in range(start, end, batch_size):
            bend = min(end, bstart + batch_size)
            dense = vec[bstart:bend].toarray().astype(np.float32)
            f.write(dense.tobytes(order='C'))
    shard_info = {
        "shard": shard_name,
        "url": f"/static/{shard_name}",  # set as you will serve it
        "start_index": int(start),
        "count": int(count),
        "size_bytes": os.path.getsize(shard_path)
    }
    manifest["shards"].append(shard_info)
    start = end
    shard_idx += 1

manifest_path = os.path.join(out_dir, "manifest.json")
with open(manifest_path, "w", encoding="utf-8") as mf:
    json.dump(manifest, mf, indent=2)

print("Done. Manifest saved to", manifest_path)

Writing shard 0: vectors 0..8387 -> vectors_000.f32 (8388 vectors)
Writing shard 1: vectors 8388..16775 -> vectors_001.f32 (8388 vectors)
Writing shard 2: vectors 16776..25163 -> vectors_002.f32 (8388 vectors)
Writing shard 3: vectors 25164..33551 -> vectors_003.f32 (8388 vectors)
Writing shard 4: vectors 33552..41939 -> vectors_004.f32 (8388 vectors)
Writing shard 5: vectors 41940..50327 -> vectors_005.f32 (8388 vectors)
Writing shard 6: vectors 50328..58715 -> vectors_006.f32 (8388 vectors)
Writing shard 7: vectors 58716..67103 -> vectors_007.f32 (8388 vectors)
Writing shard 8: vectors 67104..75491 -> vectors_008.f32 (8388 vectors)
Writing shard 9: vectors 75492..80471 -> vectors_009.f32 (4980 vectors)
Done. Manifest saved to out/manifest.json
